In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter
from scipy import misc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Image
imagePath = 'details.jpg'
image = Image.open(imagePath)
grayscaleImage = image.convert('F')
plt.imshow(grayscaleImage, cmap='gray')
plt.show()

In [ ]:
# Variables
sigma = 1.6
s = 2
k = 2 ** (1/s)
noOfOctaves = 4
imagesPerOctave = s + 3

In [ ]:
# Initialize arrays of values which will be used as a blur parameter to the gaussian filter
blurValues = np.zeros((noOfOctaves, imagesPerOctave))
val = sigma/2
for i in range(noOfOctaves):
    for j in range(imagesPerOctave):
        blurValues[i][j] = val
        val = val*k
    val = blurValues[i][2]
blurValues

In [ ]:
# Set up images of different scales for each octave
Images = []
ImageSize2 = np.array(grayscaleImage) # original-size image
ImageSize1 = misc.imresize(ImageSize2, 200, 'bilinear')
ImageSize4 = misc.imresize(ImageSize1, 50, 'bilinear')
ImageSize8 = misc.imresize(ImageSize2, 50, 'bilinear')
Images.append(ImageSize1)
Images.append(ImageSize2)
Images.append(ImageSize4)
Images.append(ImageSize8)

fig = plt.figure(figsize=(20, 20))
for i in range(len(Images)):
    fig.add_subplot(1, 4, i+1)
    plt.imshow(Images[i], cmap='gray')
plt.show()

In [ ]:
# Calculate Gaussian blurred images for each octave
octaves = []
for i in range(noOfOctaves):
    octaves.append(np.zeros((Images[i].shape[0], Images[i].shape[1], imagesPerOctave)))

for i in range(noOfOctaves):
    for j in range(imagesPerOctave):
        octaves[i][:,:,j]= gaussian_filter(Images[i], sigma=blurValues[i][j])

In [ ]:
# Calculate differences of Gaussians(DoG) for each octave
DoGs = []
DoGImageNo = imagesPerOctave-1

for i in range(DoGImageNo):
    DoGs.append(np.zeros((Images[i].shape[0], Images[i].shape[1], DoGImageNo)))

for i in range(noOfOctaves):
    for j in range(DoGImageNo):
        DoGs[i][:,:,j] = octaves[i][:,:,j+1] - octaves[i][:,:,j]

In [ ]:
plt.imshow(DoGs[0][:,:,0], cmap='gray')
plt.show()

In [ ]:
# Locate extrema in DoG images
# Filter the extrema, removing keypoints along edges
contrastThreshold = 0.03
keypoints = []
eigenValMagRatio = 10

for octave in range(noOfOctaves):
    for i in range(1, DoGImageNo - 1):
        for y in range(1, DoGs[octave][:,:,i].shape[0] - 1):
            for x in range(1, DoGs[octave][:,:,i].shape[1] - 1):
                currPixVal = DoGs[octave][y, x, i]
                if np.abs(currPixVal / 255)  < contrastThreshold:
                    continue
                isMin = True
                isMax = True
                for r in range(-1, 2):
                    for q in range(-1, 2):
                        for p in range(-1, 2):
                            if p==0 and q==0 and r==0:
                                continue
                            neighbour = DoGs[octave][y+q, x+p, i+r]
                            if neighbour <= currPixVal:
                                isMin = False
                            if neighbour >= currPixVal:
                                isMax = False
                            if not isMax and not isMin:
                                break
                        if not isMax and not isMin:
                            break
                    if not isMax and not isMin:
                        break
                if isMax or isMin:
                    Dxx = DoGs[octave][y, x+1, i] - 2*DoGs[octave][y, x, i] + DoGs[octave][y, x-1, i]
                    Dyy = DoGs[octave][y+1, x, i] - 2*DoGs[octave][y, x, i] + DoGs[octave][y-1, x, i]
                    Dxy = (DoGs[octave][y+1, x+1, i] - DoGs[octave][y-1, x+1, i] - DoGs[octave][y+1, x-1, i] + DoGs[octave][y-1, x-1, i]) / 4
                    trace = Dxx + Dyy
                    det = Dxx*Dyy - Dxy**2
                    isKeyPoint = (trace**2 / det) / 255 < ((r + 1)**2 / r)

                    if isKeyPoint:
                        keypoint = ((x,y), octave, blurValues[octave][i], 'min' if isMin else 'max', currPixVal)
                        keypoints.append(keypoint)


In [ ]:
# Test area
bl0 = [j for j in keypoints if j[1] == 0]
bl1 = [j for j in keypoints if j[1] == 1]
bl2 = [j for j in keypoints if j[1] == 2]
bl3 = [j for j in keypoints if j[1] == 3]


plt.imshow(ImageSize2, cmap='gray')
x = [i[0][0] for i in bl1]
y = [i[0][1] for i in bl1]
plt.plot(x, y, 'bo', markersize=3)
plt.show()
len(bl0), len(bl1), len(bl2), len(bl3)

In [ ]:
# keypoint magnitude and orientation assignment
# Find the DoG which has the nearest blur value compared to that of the current keypoint
magnitudes = []
for i in range(noOfOctaves):
    magnitudes.append(np.zeros((Images[i].shape[0], Images[i].shape[1], DoGImageNo)))

orientations = []
for i in range(noOfOctaves):
    orientations.append(np.zeros((Images[i].shape[0], Images[i].shape[1], DoGImageNo)))

for i in range(noOfOctaves):
    for j in range(DoGImageNo):
        for y in range(1, DoGs[i][:,:,j].shape[0] - 1):
            for x in range(1, DoGs[i][:,:,j].shape[1] - 1):
                mag = np.sqrt(DoGs[i][y, x+1, j] - DoGs[i][y, x-1, j])**2 + (DoGs[i][y+1, x, j] - DoGs[i][y-1, x, j])**2
                ori = np.arctan((DoGs[i][y+1, x, j] - DoGs[i][y-1, x, j]) / (DoGs[i][y, x+1, j] - DoGs[i][y, x-1, j]))
                magnitudes[i][y, x, j] = mag
                orientations[i][y, x, j] = ori

In [ ]:
# Keypoint descriptor calculation